In [1]:
import pandas as pd
import glob

# Dimensão acesso

files = glob.glob('../Dados/Raw/Dimensão Acesso/*.csv')
bacen = pd.DataFrame(columns=['Date'])

for file in files:
    df = pd.read_csv(file, sep=";", encoding='latin-1', decimal = '.', thousands = ',')
    bacen = bacen.merge(df, how='outer', left_on='Date', right_on='Date')

bacen = bacen.drop([16])
regioes = ['Central-West','Northeast','North ','Southeast','South ']
troca_nomes = pd.read_excel('../Dados/Raw/Dimensão Acesso/Depara.xlsx')

dataframe = pd.DataFrame()

for regiao in regioes:
    q = bacen[bacen.columns[bacen.columns.str.contains(regiao)] | bacen.columns[bacen.columns.str.contains('Date')]]
    columns = q.columns

    for c in columns:
        for i,x in troca_nomes.iterrows():
            if (x['parte importante'] in c):
                #print(c)
                q = q.rename(columns={c:x['novo nome']})
                break
                
    q['Região'] = regiao
    dataframe = dataframe.append(q)

dataframe = dataframe.reset_index(drop = True)
dataframe = dataframe.apply(lambda x: x.str.replace(',',''))

for c in dataframe.columns:
    dataframe.loc[dataframe[c] == '-', c] = None
    
dataframe['Adults with a banking relationship (million)'] = dataframe['Adults with a banking relationship (million)'].astype(float)

dataframe[dataframe.columns[0:-2]] = dataframe[dataframe.columns[0:-2]].astype(float)

dataframe.loc[dataframe['Região'] == 'South ', 'Região'] = 'South'
dataframe.loc[dataframe['Região'] == 'North ', 'Região'] = 'North'

dataframeBrasil = dataframe.groupby(['Date']).sum().reset_index()
dataframeBrasil['Região'] = 'Brazil'

dataframe = dataframe.append(dataframeBrasil)

## Fintechs

fintAcum = pd.read_excel('../Dados/Raw/Fintechs/numero_fintechs_br.xlsx')
fintAcum = fintAcum[['Ano', 'Acumulado']]
fint = pd.read_csv('../Dados/Raw/Fintechs/fintechs.csv', sep=",", encoding='latin-1', decimal = ',', thousands = '.')
fint = fint.merge(fintAcum, how = 'outer')
fint = fint.rename(columns = {'Acumulado' : 'Number Fintechs Acumulado', 'Ano' : 'Date'})

dataframe.Date = dataframe.Date.astype(int) # modificar tipo de dado da coluna Date

dataframe = dataframe.merge(fint, left_on = 'Date', right_on = 'Date', how = 'outer')

dataframe = dataframe.sort_values(by = ['Date'])

## Deflacionar Dólar

cpi = pd.read_excel('../Dados/Raw/PCI/cpi2000_2019.xlsx')
cpi = cpi.rename(columns={'Ano':'Date',100:'Taxa CPI'})
cpi = cpi[['Date','Taxa CPI']]

taxa2019 = cpi.loc[cpi.Date == 2019, 'Taxa CPI'].values[0]

dataframe = dataframe.merge(cpi)
dataframe['Total invesment activity'] = dataframe['Total invesment activity'] * taxa2019 / dataframe['Taxa CPI']

### Celulares

celular = pd.read_excel('../Dados/Raw/Celular/Posse.xlsx')

celular = pd.melt(celular, id_vars = ['Date']).rename(columns = {'variable' : 'Região', 'value' : 'Celular'})

dataframe = dataframe.merge(celular, how = 'outer')

dataframe['Celular'] = dataframe['Celular'] * 0.01

dataframe['Total invesment activity - Celular'] = dataframe['Total invesment activity'] * dataframe['Celular']
dataframe['Number Fintechs Brazil - Celular'] = dataframe['Number Fintechs Brazil'] * dataframe['Celular']

## Area Regiao

area = pd.read_excel('../Dados/Raw/Area Regiao/area_regiao.xlsx')

dataframe = dataframe.merge(area, how = 'outer')

## Populacao Regiao

pop = pd.read_excel('../Dados/Raw/Populacao/populacao_ibge_regiao (1).xlsx')

pop = pd.melt(pop, id_vars = ['Região']).rename(columns = {'Região' : 'Date', 'variable' : 'Região', 'value' : 'População'})

dataframe = dataframe.merge(pop, how = 'outer')

dfAcesso = dataframe.copy().drop(columns = ['Taxa CPI', 'Adults with a banking relationship (million)', 'Celular']).reset_index(drop = True)

## Geografico

columns = ['Number of headquarters of authorized financial institutions',
       'Number of branches of financial institutions',
       'Number of banking service outposts of financial institutions',
       'Number of electronic service outposts of financial institutions',
       'Number of correspondents of financial institutions ',
       'Number of ATMs of financial institutions',
       'Number of POSs of financial institutions',
       'Number of authorized credit unions ',
       'Number of credit unions banking service outposts',
       'Number of credit unions electronic service outposts',
       'Number of credit unions correspondents',
       'Number of branches of finance companies',
       'Number of banking services outpost of finance companies',
       'Number of electronic service outposts of finance companies',
       'Number of microentrepreneur, and small business credit companies correspondents',
       'Total invesment activity', 'Number Fintechs Brazil',
       'Number Fintechs Acumulado', 'Total invesment activity - Celular',
       'Number Fintechs Brazil - Celular']

## Geografico

dfGeografico = pd.DataFrame()

for c in columns:
    dfGeografico[c + '_Geo'] = dfAcesso[c] * 1000 / dfAcesso['Área']
    
dfGeografico = pd.concat([dfAcesso[['Date', 'Região']], dfGeografico, dfAcesso[['Total invesment activity', 'Number Fintechs Brazil',
       'Total invesment activity - Celular',
       'Number Fintechs Brazil - Celular']]], axis = 1)

## Demográfico

dfDemografico = pd.DataFrame()

for c in columns:
    dfDemografico[c + '_Demo'] = dfAcesso[c] * 10000 / dfAcesso['População']
    
dfDemografico = pd.concat([dfAcesso[['Date', 'Região']], dfDemografico, dfAcesso[['Total invesment activity', 'Number Fintechs Brazil',
       'Total invesment activity - Celular',
       'Number Fintechs Brazil - Celular']]], axis = 1)

# Dimensão Uso

dfUso = dataframe[['Região', 'Date', 'População', 'Adults with a banking relationship (million)']]

dfUso['Adults with a banking relationship (million)'] = dfUso['Adults with a banking relationship (million)'] * 1000000

dfUso['Bancarização'] = dfUso.apply(lambda row : row['Adults with a banking relationship (million)'] / row['População'], axis = 1)

# PIB

names = ['Date', 'Central-West', 'Northeast', 'North', 'Southeast', 'South', 'Brazil']

path = '../Dados/Raw/PIB/PIB 2010 A 2017.xlsx'

pib = pd.read_excel(path, parse_dates = ['Date'], names = names, header = 0)

pib.Date = pib.Date.dt.year
pib = pd.melt(pib, id_vars = ['Date']).rename(columns = {'value' : 'PIB', 'variable' : 'Região'})

dfUso = dfUso.merge(pib)

## Depósito - Crédito

depCred = pd.read_csv('../Dados/Raw/Depositos/Consolidado.csv', sep=",", encoding='latin-1', header = 0) 

columnsDep = ['VERBETE_420_DEPOSITOS_DE_POUPANCA',
           'VERBETE_430_DEPOSITOS_INTERIFNANCEIROS',
           'VERBETE_431_DEPOSITOS_INTERFINANCEIROS',
           'VERBETE_432_DEPOSITOS_A_PRAZO',
           'VERBETE_112_DEPOSITOS_BANCARIOS']

columnsCred = ['VERBETE_160_OPERACOES_DE_CREDITO', 'VERBETE_172_OUTROS_CREDITOS']

depCred['Depósito'] = 0
depCred['Crédito' ] = 0

for c in columnsDep:
    depCred['Depósito'] = depCred['Depósito'] + depCred[c]

for c in columnsCred:
    depCred['Crédito'] = depCred['Crédito'] + depCred[c]

depCred = depCred.rename(columns = {'#DATA_BASE' : 'Date'})
depCred = depCred[['Date', 'UF', 'Crédito', 'Depósito']]

depCred.Date = depCred.Date.astype(int).astype(str)

depCred['Year']  = depCred.Date.apply(lambda date : date[0:4])
depCred['Month'] = depCred.Date.apply(lambda date : date[4:6])

depCred = depCred.drop(columns = ['Date'])
depCred = depCred.groupby(['UF', 'Year']).sum().reset_index()

estadoRegiao = pd.read_csv('../Dados/Raw/EstadoRegiao/Sigla_estado.csv', encoding = 'latin_1')
estadoRegiao = estadoRegiao.rename(columns = {'Sigla' : 'UF'})

depCred = depCred.merge(estadoRegiao)
depCred = depCred.drop(columns = {'UF'})
depCred = depCred.rename(columns = {'Year' : 'Date'})
depCred = depCred.groupby(['Região', 'Date']).sum().reset_index()
depCred.Date = depCred.Date.astype(int)

depCredBrasil = depCred.groupby(['Date']).sum().reset_index()

depCredBrasil['Região'] = 'Brazil'

depCred = depCred.append(depCredBrasil)

dfUso = dfUso.merge(depCred)

## Nível de Crédito e Depósito

dfUso['Nível Crédito'] = dfUso['Crédito'] / dfUso['PIB']

dfUso['Nível Depósito'] = dfUso['Depósito'] / dfUso['PIB']

dfUso = dfUso[['Date', 'Região', 'Bancarização', 'Nível Crédito', 'Nível Depósito']]

# Exportar

dfDemografico.to_csv('../Dados/Processed/Acesso/Demografico.csv', index = None)
dfGeografico .to_csv( '../Dados/Processed/Acesso/Geofrafico.csv', index = None)

dfUso.to_csv('../Dados/Processed/Uso/Consolidado.csv', index = None)

